### running the evaluation for the challenge dataset

In [1]:
%load_ext autoreload

In [2]:
%autoreload

In [3]:
import torch
import torch.nn as nn
import numpy as np
import os
import matplotlib.pyplot as plt
from torchinfo import summary
from collections import defaultdict, namedtuple
import pandas as pd
# import proplot as pplt
import warnings
warnings.filterwarnings("ignore") # I should only put this in place when I am doing plotting and using proplot....
from twaibrain.braintorch.utils.resize import crop_or_pad_dims
from twaibrain.brainpreprep.utils.image_io import load_image
import scipy
import scipy.stats

# model architecture
import json
from twaibrain.braintorch.models.nnUNet.nnUNetV2_model_loader import get_network_from_plans
from twaibrain.braintorch.models.ssn import SSN_Wrapped_Deep_Supervision, SSN_Wrapped_Deep_Supervision_LLO, Hierarchical_SSN_with_ConvRefine, Hierarchical_SSN_with_ConvSpatialAttention

# fitting code
from twaibrain.braintorch.fitting_and_inference.get_trainer import get_trainer
from twaibrain.braintorch.fitting_and_inference.get_scratch_dir import scratch_dir
from twaibrain.braintorch.fitting_and_inference.optimizer_constructor import OptimizerConfigurator
from twaibrain.braintorch.fitting_and_inference.lightning_fitter import StandardLitModelWrapper

# loss function
from twaibrain.braintorch.losses.ssn_losses import DeepSupervisionSSN, SSNCombinedDiceXent_and_MC_loss, SSNCombinedDiceXent_and_MC_loss_FromSamples
from twaibrain.braintorch.losses.generic_deep_supervision import MultiDeepSupervisionLoss, DeepSupervisionLoss
from twaibrain.braintorch.losses.dice_loss import SoftDiceV2
from twaibrain.braintorch.losses.xent import dice_xent_loss

# data
from twaibrain.brainexperiments.run_nnUNet_v2.old_dataloading.dataset_pipelines import load_data
from twaibrain.braintorch.data.legacy_dataset_types.dataset_wrappers import MonaiAugmentedDataset
from twaibrain.braintorch.augmentation.nnunet_augmentations import get_nnunet_transforms, get_val_transforms
from torch.utils.data import ConcatDataset
from twaibrain.braintorch.data.legacy_dataset_types.mri_dataset_inram import MRISegmentation3DDataset
from twaibrain.braintorch.data.legacy_dataset_types.mri_dataset_from_file import MRISegmentationDatasetFromFile, ArrayMRISegmentationDatasetFromFile
from twaibrain.braintorch.data.legacy_dataset_types.mri_dataset_directory_parsers import *

# evaluation code
from twaibrain.brainexperiments.run_nnUNet_v2.evaluation.eval_helper_functions import *
from twaibrain.brainexperiments.run_nnUNet_v2.evaluation.model_predictions import get_means_and_samples_2D, ssn_ensemble_mean_and_samples
from twaibrain.brainexperiments.run_nnUNet_v2.evaluation.model_predictions import *

### setting up params for the rest of the code to run

In [4]:
ARGS = namedtuple("args", "dataset test_split val_split seed " + 
                  "empty_slice_retention batch_size cross_validate cv_split cv_test_fold_smooth no_test_fold " +
                  "num_workers dice_factor xent_factor xent_reweighting eval_split uncertainty_type ckpt_dir"
)

In [5]:
VOXELS_TO_WMH_RATIO = 382
VOXELS_TO_WMH_RATIO_EXCLUDING_EMPTY_SLICES = 140
ESR = 0.5


# setup xent reweighting factor
XENT_VOXEL_RESCALE = VOXELS_TO_WMH_RATIO - (1-ESR) * (VOXELS_TO_WMH_RATIO - VOXELS_TO_WMH_RATIO_EXCLUDING_EMPTY_SLICES)

XENT_WEIGHTING = XENT_VOXEL_RESCALE/2

In [6]:
args = ARGS(
    dataset = "chalfull",
    
    test_split=0.15,
    val_split=0.15,
    eval_split='all',

    seed=5,

    empty_slice_retention=ESR,

    uncertainty_type="ssn",

    batch_size=2,
    cross_validate=True,
    cv_split=0,
    cv_test_fold_smooth=1,
    no_test_fold="false",
    num_workers=16,

    dice_factor=1,
    xent_factor=1,
    xent_reweighting=XENT_WEIGHTING,
    ckpt_dir= "/home/s2208943/projects/twaibrain/twaibrain/brainexperiments/run_nnUNet_v2/training/ssn_model_ckpts/"
)

### load a model checkpoint and get relevant predictions

In [7]:
model_config = "/home/s2208943/projects/twaibrain/twaibrain/braintorch/models/nnUNet/cvd_configs/nnUNetResEncUNetMPlans.json"

with open(model_config) as f:
    model_config = json.load(f)

In [8]:
dims = "2d"
config = model_config['configurations'][dims]['architecture']
network_name = config['network_class_name']
kw_requires_import = config['_kw_requires_import']

def get_model():
    return get_network_from_plans(
        arch_class_name=network_name,
        arch_kwargs=config['arch_kwargs'],
        arch_kwargs_req_import=kw_requires_import,
        input_channels=3,
        output_channels=32,
        allow_init=True,
        deep_supervision=True,
    )

model = get_model()

In [9]:
ssn_config = {
    'intermediate_channels':32,
    'out_channels':2,
    'dims':2,
    'rank':25,
    'diagonal':False,
}

ssn_model = SSN_Wrapped_Deep_Supervision(model, 5, ssn_config)

In [10]:
ensemble_models = [ssn_model] + [SSN_Wrapped_Deep_Supervision(get_model(), 5, ssn_config) for _ in range(9)]

In [11]:
%autoreload

### storing the model outputs

In [12]:
from twaibrain.braintorch.data.legacy_dataset_types import LimitedSizeDataset

In [13]:
def evaluate_dataset(model, full_ds, ds_ids, output_dir, model_name, model_func):
    start = 0
    end = 10
    
    while start < len(full_ds):
        ds = LimitedSizeDataset(full_ds, start=start, end=end)
        IDs = ds_ids[start:end]
    
        start = end
        end += 10
    
        # get predictions
        means, samples, miscs = get_means_and_samples_2D(model, ds, 10, model_func, args=args, resize=True)
        # means, samples, misc = get_nnunet_means_and_samples(model_raw, ds, num_samples=10, model_func_name=model_func_name, args=args)
        means = [m.squeeze(0).swapaxes(0, 1)[:,:2] for m in means]

        if samples[0] is not None:
            samples = [s.swapaxes(2, 3) for s in samples]
            samples = [s[:,:,:,:2].squeeze() for s in samples]
        
        # get uncertainty maps
        ent_maps = get_uncertainty_maps(means, samples, miscs, args)
    
        # write results
        write__output_map_results_to_disk(args, IDs, ent_maps, means, model_name=model_name, output_dir=output_dir)

        # break


In [14]:
mss3_parser = MSS3MultiRaterDataParser(
    # paths on the cluster for the in house data
    "/home/s2208943/datasets/Inter_observer",
    "/home/s2208943/preprocessed_data/MSS3_InterRaterData"
)

mss3_ds = ArrayMRISegmentationDatasetFromFile(mss3_parser) 

mss3_ids = [out[2] for out in mss3_ds]

mss3_output_dir = "/home/s2208943/preprocessed_data/MSS3_InterRaterData/output_maps/nnunet/"

In [15]:
domains_chal_train = ["training_Singapore", "training_Utrecht", "training_Amsterdam_GE3T"]
    
chal_ds_train = ConcatDataset([
    MRISegmentation3DDataset("/home/s2208943/preprocessed_data/WMHChallenge_InterRaterData/collated", no_labels=True, xy_only=False, domain_name=dn)
    for dn in domains_chal_train
])

chal_train_ids = [out[2] for out in chal_ds_train]

chal_train_output_dir = "/home/s2208943/preprocessed_data/WMHChallenge_InterRaterData/output_maps/training/nnunet/"

In [16]:
domains_chal_test = ["test_Amsterdam_GE1T5", "test_Amsterdam_Philips_VU_PETMR_01", "test_Utrecht", "test_Amsterdam_GE3T", "test_Singapore"]
    
chal_ds_test = ConcatDataset([
    MRISegmentation3DDataset("/home/s2208943/preprocessed_data/WMHChallenge_InterRaterData/collated", no_labels=True, xy_only=False, domain_name=dn)
    for dn in domains_chal_test
])

chal_test_ids = [out[2] for out in chal_ds_test]

chal_test_output_dir = "/home/s2208943/preprocessed_data/WMHChallenge_InterRaterData/output_maps/test/nnunet/"

In [17]:
ADNI_ds = MRISegmentation3DDataset("/home/s2208943/preprocessed_data/ADNI300/collated", no_labels=True, xy_only=False)
adni_ids = [out[2] for out in ADNI_ds]
adni_out_dir = "/home/s2208943/preprocessed_data/ADNI300/ADNI_300_output_maps/nnunet/"

In [18]:
%autoreload

### evaluating the X2 version of SSN-Ens

In [19]:
base_folder = "/home/s2208943/projects/twaibrain/twaibrain/brainexperiments/run_nnUNet_v2/training/ssn_model_ckpts_redo/"

# some of the runs got corrupted when they "upgraded" the fileserver, hence doing this....
loaded_ensemble_models = []
for m in range(10):
    success = False
    cv_split=0
    while cv_split < 6:
        try:
            model_dir = os.path.join(base_folder, f"nnunet2D_ssnX2_ens{m}_cv{cv_split}")  
            print("model dir: ", model_dir)
            ckpt = sorted([f for f in os.listdir(model_dir) if f.endswith(".ckpt")])[-1]
            loaded_ensemble_models.append(load_best_checkpoint(ensemble_models[m], None, None, None, os.path.join(model_dir, ckpt)).model.cuda())
            success = True
            break
        except:
            cv_split += 1

    print(f"m: {m} suceeded? {success}")
        


model dir:  /home/s2208943/projects/twaibrain/twaibrain/brainexperiments/run_nnUNet_v2/training/ssn_model_ckpts_redo/nnunet2D_ssnX2_ens0_cv0
m: 0 suceeded? True
model dir:  /home/s2208943/projects/twaibrain/twaibrain/brainexperiments/run_nnUNet_v2/training/ssn_model_ckpts_redo/nnunet2D_ssnX2_ens1_cv0
m: 1 suceeded? True
model dir:  /home/s2208943/projects/twaibrain/twaibrain/brainexperiments/run_nnUNet_v2/training/ssn_model_ckpts_redo/nnunet2D_ssnX2_ens2_cv0
m: 2 suceeded? True
model dir:  /home/s2208943/projects/twaibrain/twaibrain/brainexperiments/run_nnUNet_v2/training/ssn_model_ckpts_redo/nnunet2D_ssnX2_ens3_cv0
m: 3 suceeded? True
model dir:  /home/s2208943/projects/twaibrain/twaibrain/brainexperiments/run_nnUNet_v2/training/ssn_model_ckpts_redo/nnunet2D_ssnX2_ens4_cv0
m: 4 suceeded? True
model dir:  /home/s2208943/projects/twaibrain/twaibrain/brainexperiments/run_nnUNet_v2/training/ssn_model_ckpts_redo/nnunet2D_ssnX2_ens5_cv0
m: 5 suceeded? True
model dir:  /home/s2208943/project

In [20]:
evaluate_dataset(loaded_ensemble_models, ADNI_ds, adni_ids, adni_out_dir, "nnunetX2SsnEns2D", ssn_ensemble_mean_and_samples)

100%|███████████████████████████████████████████████████████████████| 10/10 [00:26<00:00,  2.60s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:21<00:00,  2.11s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11.13it/s]


...

100%|███████████████████████████████████████████████████████████████| 10/10 [00:21<00:00,  2.12s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11.42it/s]


folder already exists


100%|███████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.72it/s]


In [21]:
evaluate_dataset(loaded_ensemble_models, chal_ds_test, chal_test_ids, chal_test_output_dir, "nnunetX2SsnEns2D", ssn_ensemble_mean_and_samples)

100%|███████████████████████████████████████████████████████████████| 10/10 [00:42<00:00,  4.25s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:34<00:00,  3.41s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.56it/s]


...

100%|███████████████████████████████████████████████████████████████| 10/10 [00:19<00:00,  1.93s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11.56it/s]


folder already exists


100%|███████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.78it/s]


In [22]:
evaluate_dataset(loaded_ensemble_models, chal_ds_train, chal_train_ids, chal_train_output_dir, "nnunetX2SsnEns2D", ssn_ensemble_mean_and_samples)

100%|███████████████████████████████████████████████████████████████| 10/10 [00:19<00:00,  1.93s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:19<00:00,  1.93s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11.97it/s]


...

100%|███████████████████████████████████████████████████████████████| 10/10 [00:33<00:00,  3.38s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.52it/s]


folder already exists


100%|███████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.39it/s]


In [23]:
evaluate_dataset(loaded_ensemble_models, mss3_ds, mss3_ids, mss3_output_dir, "nnunetX2SsnEns2D", ssn_ensemble_mean_and_samples)

100%|███████████████████████████████████████████████████████████████| 10/10 [00:24<00:00,  2.44s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:24<00:00,  2.45s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.79it/s]


...

 80%|███████████████████████████████████████████████████▏            | 8/10 [00:19<00:04,  2.41s/it]


ssn
generating uncertainty maps


100%|█████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  6.65it/s]


folder already exists


100%|█████████████████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.15it/s]


### Evaluating the X2 version of SSN

In [24]:
evaluate_dataset(loaded_ensemble_models[0], ADNI_ds, adni_ids, adni_out_dir, "nnunetX2Ssn2D", ssn_mean_and_samples)

100%|███████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.24it/s]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.29it/s]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 10.85it/s]


...

100%|███████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.32it/s]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11.25it/s]


folder already exists


100%|███████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.94it/s]


In [20]:
loaded_ensemble_models[0].ssn_heads[0].return_cpu_dist = True

In [21]:
evaluate_dataset(loaded_ensemble_models[0], chal_ds_test, chal_test_ids, chal_test_output_dir, "nnunetX2Ssn2D", ssn_mean_and_samples)

100%|███████████████████████████████████████████████████████████████| 10/10 [00:46<00:00,  4.60s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:32<00:00,  3.22s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  9.72it/s]


...

100%|███████████████████████████████████████████████████████████████| 10/10 [00:17<00:00,  1.73s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 10.40it/s]


folder already exists


100%|███████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.80it/s]


In [22]:
evaluate_dataset(loaded_ensemble_models[0], chal_ds_train, chal_train_ids, chal_train_output_dir, "nnunetX2Ssn2D", ssn_mean_and_samples)

100%|███████████████████████████████████████████████████████████████| 10/10 [00:17<00:00,  1.71s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:17<00:00,  1.71s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 17.24it/s]


...

100%|███████████████████████████████████████████████████████████████| 10/10 [00:30<00:00,  3.05s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.15it/s]


folder already exists


100%|███████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.65it/s]


In [23]:
evaluate_dataset(loaded_ensemble_models[0], mss3_ds, mss3_ids, mss3_output_dir, "nnunetX2Ssn2D", ssn_mean_and_samples)

100%|███████████████████████████████████████████████████████████████| 10/10 [00:19<00:00,  1.91s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:18<00:00,  1.87s/it]


ssn
generating uncertainty maps


100%|███████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.59it/s]


...

 80%|███████████████████████████████████████████████████▏            | 8/10 [00:15<00:03,  1.89s/it]


ssn
generating uncertainty maps


100%|█████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  6.75it/s]


folder already exists


100%|█████████████████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.23it/s]
